In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
df = pd.read_csv("/users/amulya/Downloads/HousePricePrediction.csv")
unwanted_columns = ['Id','YearBuilt','YearRemodAdd']
df['built_age'] = 2024 - df['YearBuilt']
df['remodel_age'] = 2024 - df['YearRemodAdd']
# print(df['remodel_age'])
df['OverallCond'] = df['OverallCond'].replace({1 : 4,2:4,3:4,8:9})
print(df['OverallCond'].value_counts())
a = df.isnull().sum()
print(a)
df = df.dropna()
# plt.bar(df['OverallCond'],df['SalePrice'])
# plt.show

OverallCond
5    1645
6     531
7     390
9     185
4     168
Name: count, dtype: int64
Id                 0
MSSubClass         0
MSZoning           4
LotArea            0
LotConfig          0
BldgType           0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
Exterior1st        1
BsmtFinSF2         1
TotalBsmtSF        1
SalePrice       1459
built_age          0
remodel_age        0
dtype: int64


In [10]:
df = df.drop(unwanted_columns,axis = 1)
dummies = pd.get_dummies(df['MSZoning'])
df = pd.concat([df, dummies], axis=1)
LotConfig_dummies = pd.get_dummies(df['LotConfig'])
df = pd.concat([df, LotConfig_dummies], axis=1)
BldgType_dummies = pd.get_dummies(df['BldgType'])
df = pd.concat([df, BldgType_dummies], axis=1)
Exterior1st_dummies = pd.get_dummies(df['Exterior1st'])
df = pd.concat([df, Exterior1st_dummies], axis=1)
MSSubClass_dummies = pd.get_dummies(df['MSSubClass'])
df = pd.concat([df, MSSubClass_dummies], axis=1)
OverallCond_dummies = pd.get_dummies(df['OverallCond'])
df = pd.concat([df, OverallCond_dummies], axis=1)
print(df)


      MSSubClass MSZoning  LotArea LotConfig BldgType  OverallCond  \
0             60       RL     8450    Inside     1Fam            5   
1             20       RL     9600       FR2     1Fam            9   
2             60       RL    11250    Inside     1Fam            5   
3             70       RL     9550    Corner     1Fam            5   
4             60       RL    14260       FR2     1Fam            5   
...          ...      ...      ...       ...      ...          ...   
1455          60       RL     7917    Inside     1Fam            5   
1456          20       RL    13175    Inside     1Fam            6   
1457          70       RL     9042    Inside     1Fam            9   
1458          20       RL     9717    Inside     1Fam            6   
1459          20       RL     9937    Inside     1Fam            6   

     Exterior1st  BsmtFinSF2  TotalBsmtSF  SalePrice  ...     90    120  \
0        VinylSd         0.0        856.0   208500.0  ...  False  False   
1        

In [11]:
duplicate_columns = ['MSZoning','LotConfig','BldgType','Exterior1st','MSSubClass','OverallCond']
df = df.drop(duplicate_columns,axis = 1)
print(df)

      LotArea  BsmtFinSF2  TotalBsmtSF  SalePrice  built_age  remodel_age  \
0        8450         0.0        856.0   208500.0         21           21   
1        9600         0.0       1262.0   181500.0         48           48   
2       11250         0.0        920.0   223500.0         23           22   
3        9550         0.0        756.0   140000.0        109           54   
4       14260         0.0       1145.0   250000.0         24           24   
...       ...         ...          ...        ...        ...          ...   
1455     7917         0.0        953.0   175000.0         25           24   
1456    13175       163.0       1542.0   210000.0         46           36   
1457     9042         0.0       1152.0   266500.0         83           18   
1458     9717      1029.0       1078.0   142125.0         74           28   
1459     9937       290.0       1256.0   147500.0         59           59   

      C (all)     FV     RH    RL  ...     90    120    160    180    190  

In [12]:
df.columns = df.columns.astype(str)
price = df['SalePrice']
# print(price)
min = df['SalePrice'].min()
max = df['SalePrice'].max()
print(min,max)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df)
# print(X_scaled.info())
df_scaled = pd.DataFrame(X_scaled, columns=df.columns)
x = df_scaled.drop(['SalePrice'], axis= 1)
y = df_scaled['SalePrice']
# Y_scaled = scaler.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
ridge = Ridge(alpha=0.6, max_iter=500, tol=0.1, fit_intercept=True, random_state=42)
ridge.fit(x_train, y_train)
x_pred_ridge_test = ridge.predict(x_test)
r2_score(y_test, x_pred_ridge_test)
mae = mean_absolute_error(y_test, x_pred_ridge_test)
mse = mean_squared_error(y_test, x_pred_ridge_test)
r_squared = r2_score(y_test, x_pred_ridge_test)
rmse = np.sqrt(mse)
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R²):", r_squared)  
print("Root Mean Squared Error (RMSE):", rmse)
predicted_price = ((min+max)*x_pred_ridge_test)+min
# print(predicted_price)
ridge_reg_dataset = x_test.copy()
ridge_reg_dataset['Actual SalePrice'] = y_test.values
ridge_reg_dataset['Predicted SalePrice'] = x_pred_ridge_test
ridge_reg_dataset['actual price'] = price
ridge_reg_dataset['predicted price'] = x_pred_ridge_test
# print(ridge_reg_dataset)


34900.0 755000.0
Mean Absolute Error (MAE): 0.043356598461174144
Mean Squared Error (MSE): 0.004539975492211864
R-squared (R²): 0.66263320934278
Root Mean Squared Error (RMSE): 0.06737934024767432


In [13]:
ridge_reg_dataset = x_test.copy()
ridge_reg_dataset['Actual SalePrice'] = y_test.values
ridge_reg_dataset['Predicted SalePrice'] = x_pred_ridge_test
ridge_reg_dataset['actual price'] = price
ridge_reg_dataset['predicted price'] = predicted_price
print(ridge_reg_dataset)

       LotArea  BsmtFinSF2  TotalBsmtSF  built_age  remodel_age  C (all)   FV  \
892   0.033252         0.0     0.173322   0.340580     0.116667      0.0  0.0   
1105  0.051209         0.0     0.239444   0.115942     0.250000      0.0  0.0   
413   0.035804         0.0     0.164975   0.601449     1.000000      0.0  0.0   
522   0.017294         0.0     0.164321   0.456522     1.000000      0.0  0.0   
1036  0.054210         0.0     0.265139   0.021739     0.033333      0.0  0.0   
...        ...         ...          ...        ...          ...      ...  ...   
331   0.032139         0.0     0.172831   0.376812     0.300000      0.0  0.0   
323   0.021127         0.0     0.190180   0.398551     0.083333      0.0  0.0   
650   0.031901         0.0     0.133061   0.021739     0.050000      0.0  1.0   
439   0.051667         0.0     0.111948   0.652174     0.166667      0.0  0.0   
798   0.057108         0.0     0.315221   0.014493     0.016667      0.0  0.0   

       RH   RL   RM  ...  1

In [14]:
from sklearn.metrics import mean_squared_error 
error = ridge_reg_dataset['actual price']-ridge_reg_dataset['predicted price']
mse_price = mean_squared_error(ridge_reg_dataset['actual price'], ridge_reg_dataset['predicted price'])
rmse_price = np.sqrt(mse_price)
print(rmse_price)


49595.63151785555


In [15]:
from sklearn.neighbors import KNeighborsRegressor 
knn = KNeighborsRegressor(n_neighbors=7) 
  
knn.fit(x_train, y_train) 
predicted_price = knn.predict(x_test)
mse = mean_squared_error(ridge_reg_dataset['actual price'], predicted_price)
rmse = np.sqrt(mse)
rmse

198445.92620270993

In [18]:
from sklearn.tree import DecisionTreeRegressor 

model = DecisionTreeRegressor(random_state=42)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
mse = mean_squared_error(ridge_reg_dataset['actual price'], y_pred)
rmse = np.sqrt(mse)
rmse

198445.9053260073